# NEURON tutorial

The NEURON simulation environment is widely used in classrooms and laboratories to build models of neurons and network of neurons.
See: https://www.neuron.yale.edu/neuron/

## The NEURON software

NEURON was original developed with a scripting language called "hoc". It is still widely used and only recently it is being replaced with a Python interface [(Hines et al., 2009)](http://journal.frontiersin.org/article/10.3389/neuro.11.001.2009/abstract). All NEURON routines are written in either C/C++ or hoc.
NEURON features also a Graphical User Interface (GUI), equipped with builders for cells and networks, channels and electrical circuits. Feel free to explore the NEURON GUI, but here we prefer writing some lines of Python code.
___
In this tutorial we will see how to use a small number of NEURON functions, while you can find further information on NEURON modeling concepts and programming-related terms in the [programmer's reference](http://www.neuron.yale.edu/neuron/static/new_doc/index.html).
____

In [ ]:
# Let's run this line to be sure that we can access the files we need
! pip -q install --upgrade "hbp-service-client==1.0.0"

In [ ]:
# import matplotlib.pyplot to plot the results and define plot size
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

# Import NEURON as a Python module, together with the hoc language and the gui
from neuron import h,gui

## Stylized tree representation of a cell
In NEURON a cell is represented as a tree of connected **sections**. A section is a portion of cell with homogeneous biophysical properties. 
In the simplest approximation, a section is a cilinder with some properties: lengh *L* (in ${\mu}m$), diameter *diam* (in ${\mu}m$), cytoplasmic resistance *Ra* (in $ohm*cm$), membrane capacity (in ${\mu}F/cm^2$), channels density.
A **segment** (or compartment), *seg*, is the simplest unit of discretization of a section. It is a numerical entity, that defines the spatial discretizion ${\delta}x = L / nseg$.

In [ ]:
# Create a section 
soma = h.Section()
soma.Ra = 1
soma.L = 100
soma.diam = 10
soma.cm = 1

## Implementing a current-clamp experiment
A stimulation electrode in NEURON is a "point process", i.e. an object attached to a precise location along a section. The location is specified as a number from 0 to 1.
To set up a current clamp experiment we need to specify its location, *delay* (time from the start of the simulation), *amp* (amplitude of the injected current, in *nA*) and *dur* (duration of stimulus in *ms*).

In [ ]:
# Inject a current step into the soma
stim = h.IClamp(soma(0.5))         # Place a stimulation electrode in the middle of the soma
stim.delay = 1
stim.amp = 1
stim.dur = 3

## Recording the results
The hoc language is equipped with "Vectors" (similar to Python lists) that allow to record many variables specified in our models. For instance, it is possible to record arrays of time and voltage at a specific location.

In [ ]:
# Initialize NEURON vectors to record time, voltage and current
rec_t = h.Vector()
rec_t.record(h._ref_t)

rec_v = h.Vector()
rec_v.record(soma(0.5)._ref_v)

rec_i = h.Vector()
rec_i.record(stim._ref_i)

## Running a simulation
Before running a simulation we have to set up some parameters: the simulation time *tstop* (in *ms*), the initial membrane voltage *v_init* in (*mV*) and the time discretization *steps_per_ms* (or *dt* = 1/steps_per_ms).

In [ ]:
# Initialize and run a simulation
h.v_init = -65
h.finitialize()
h.tstop = 5
h.steps_per_ms = 40

h.init()
h.run()

In [ ]:
# Plot the stimulation protocol and the voltage response
fig, ax1 = plt.subplots()
plt.plot(rec_t,rec_v, 'b', label = "Voltage response")
ax2 = ax1.twinx()
plt.plot(rec_t,rec_i, 'g', label = "Injected current")

ax2.set_ylim([-1,10])
ax1.set_xlabel("Time (ms)")
ax1.set_ylabel("Voltage (mV)")
ax2.set_ylabel("Current (nA)")
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

## Spatial discretization
In the previous simulation we recorded the voltage response at the middle of the soma. What would have happen if we wanted to record it just before the 1 end of the section? Try it.
Is it the result different?


To have a more refined spatial discretization we can increase *nseg*. To make sure that the electrodes have the best accuracy, it is always advisable to have an odd number of segments.
However, we always have to record from the center of each compartment, then "transpose" the recordings to visualize the results at other locations.

In [ ]:
soma.nseg = 5

def transposeRecording(reclist, nloc, tstop):
    retlist = []
    
    # each element of reclist is a neuron.h.Vector of recordings for one out of nloc spatial locations
    # (usually compartment centers) over time
    for i in range(0,int(tstop/h.dt)+1):
        retlist.append(h.Vector(nloc,0)) 
        for j in range(0,nloc):
            retlist[i].x[j] = reclist[j].x[i]
    return retlist

In [ ]:
reclist = h.List()
for i in range(0,soma.nseg):
    tmpRecVec = h.Vector()
    tmpRecVec.record(soma((0.5 + i*1.0)/soma.nseg)._ref_v)
    reclist.append(tmpRecVec)
h.init()
h.run(5)
xlocations = h.Vector()
for i in range(0,soma.nseg):
    xlocations.append(soma.L*(0.5 + i*1.0)/(soma.nseg))

spatialrec = transposeRecording(reclist, soma.nseg, 5)
fig = plt.figure()
plt.plot(xlocations, spatialrec[41]) # 1.025/0.025 = 41
plt.ylabel("Voltage (mV)")
plt.xlabel("Spatial location (um)")
plt.title("Spatial distribution at t = 1.025 ms")


The voltage variation in space it not so large.
____
__Exercise 1.__
Which parameters influence the voltage variation in space? Try to modify some values in order to:
0. increase the number of compartments;  
1. plot the distribution at t = 2.975 ms (remember that we record the time every 0.025 ms); 
2. displace the stimulation point;
3. change the biophysical parameters (length and diameter). Can you find by trial and error a combination of parameter such that the difference between *soma._ref_v(0.5)* and *soma._ref_v(0)* is roughly equal to 0.5?
___

## Biological modeling: mechanisms
Mechanisms represent ion channels, buffers, synapses dynamics in NEURON. They are not point processes but "distributed mechanisms" and they are present on the entire surface of a section. Mechanisms parameters can change within compartments.
NEURON has some built-in mechanisms, for example a passive leak current *pas* and sodium and potassium currents to generate action potentials *hh* (from the squid axon, measured by Hodgkin and Huxley!)

__Restart the kernel before continuing__

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

# Import NEURON as a Python module, together with the hoc language and the gui
from neuron import h,gui

# Insert passive and active mechanisms
hhsoma = h.Section()
hhsoma.L = 20
hhsoma.diam = 20
hhsoma.nseg = 1

hhsoma.insert('pas')
for seg in hhsoma:
    seg.pas.g = 0.003 # Conductance of the leak channels (in S/cm2)
    seg.pas.e = -60   # Leak reversal potential, it influences the steady state membrane potential
    
hhsoma.insert('hh')
for seg in hhsoma:
    seg.hh.gkbar = 0.008  # Maximal conductance of the potassium channels
    seg.hh.gnabar = 0.12  # Maximal conductance of the sodium channels

In [ ]:
stim = h.IClamp(hhsoma(0.5))
stim.delay = 1
stim.dur = 1
stim.amp = 1

time = h.Vector()
voltage = h.Vector()
time.record(h._ref_t)
voltage.record(hhsoma(0.5)._ref_v)

h.tstop = 25
h.finitialize()
h.init()
h.run()

plt.plot(time,voltage, 'k')


____
__Exercise 2.__ Which parameters influence the shape of the action potential? Modify some parameters in order to:
0. decrease the amplitude of the action potential. How can you simulate an experiment in which 95 % of sodium channels have been blocked with TTX?
1. reach a steady state voltage of -70 mV;
3. reach -75 mV during the AHP (after-hyperpolarization that follows the action potential);
4. increase the speed of the action potential; Hint: look up in the [programmer's reference](http://www.neuron.yale.edu/neuron/static/new_doc/index.html) how you can modify the temperature of a simulation. Remember that we are using Python and how to access hoc variables through Python.
___

# Detailed morphology
Neurons are represented topologically by the tree structure. Each sections has a 0 and 1 end, which can represent branching points. If we consider a section (current section), a section connected to its 0 end is the parent sections of the current section, while children sections are connected to the 1 end. A section without a parent is the "root" (usually a somatic section).

__Restart the kernel before continuing__

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

# Import NEURON as a Python module, together with the hoc language
from neuron import h

# create some sections
for sec in h.allsec():
    del sec
    
soma = h.Section(name='soma') # NEURON sections can have a name
dendrites = []
dendrites.append(h.Section(name='dend0'))

# connect a section to its parent
dendrites[0].connect(soma)

# specify which end to connect to
dendrites.append(h.Section(name='dend1'))
dendrites[1].connect(soma,1)

# ask for some characteristics
print soma.hname()
print dendrites[1].orientation()



____
__Exercise 3.__ Create a simple dendritic tree.

0. Modify the code above to create a soma and two dendrites, each connected to a different end of the soma.
1. Make one of the dendrites longer by adding ten more sections. 
____

## Morphologically-detailed models
To build a morphologically-detailed neuron model we need a reconstruction of 3D morphology, some details about its passive properties and information about the ion channels (active properties).

NEURON can load Neurolucida (and others) morphology formats through the *hoc* routines in the file "import3d.hoc".
Model of the ion channels are written in another scripting language, called *NMODL* and stored in separate .mod files.

In the following part of the tutorial we will build a neuron inspired from real thalamo-cortical cells. We will load a morphology of a real thalamo-cortical neuron (downloaded from http://neuromorpho.org/) and some of its ion channels (have a look to the .mod files). They are available on [NEURON models database](https://senselab.med.yale.edu/modeldb/).

___
__Exercise 4.__ Simulate a rebound response in a thalamo-cortical neuron. A rebound spike response can be elicited in thalamo-cortical and many other types of neurons in patch-clamp experiments with a hyperpolarizing current step.


For this exercise you will have to complete the code below. 
Before creating the neuron you have to run some cells in order to set the environment.

You will also have to restart the kernel in order to integrate the new compiled mechanisms in the class definition.
___

In [ ]:
import requests
base = 'https://github.com/NataliBarros/InSilicoNeuroscienceCourse/raw/master/'
r = requests.get(base + 'TC_HH.mod')
open('TC_HH.mod', 'wb').write(r.content)

r = requests.get(base + 'TC_iA.mod')
open('TC_iA.mod', 'wb').write(r.content)

r = requests.get(base + 'TC_Ih.mod')
open('TC_Ih.mod', 'wb').write(r.content)

r = requests.get(base + 'TC_ITGHK.mod')
open('TC_ITGHK.mod', 'wb').write(r.content)

In [ ]:
# This will compile the mechanisms (and create a new folder x86_64/special).
! nrnivmodl

If you have seen the Neuron documentation you probably know that Neuron has a GUI.
This is a way of visualizing the morphology of a neuron through the Neuron GUI.
If you have Neuron install in your local machine, you can try to run this code and
see if it works for you.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

# Import NEURON as a Python module, together with the hoc language
from neuron import h

class TC_neuron:
    """
    Model inspired from a thalamo-cortical neuron. 
    """
    def __init__(self):
        self.build_morphology()
        self.define_biophysics()
        
    def build_morphology(self):
        """
        Loads a 3D morphology (without axon) of the neuron
        """
        # Load hoc routines to import 3D morphologies
        h.load_file("import3d.hoc")
        nl = h.Import3d_SWC_read() # We have a .swc morphology file

        # load the file from Github storage
        r = requests.get(base + 'destexhe_morpho.swc')
        open('destexhe_morpho.swc', 'wb').write(r.content)

        # Read the file and creates automatically section.connect(parent) statements
        nl.input("destexhe_morpho.swc")

        # instantiate morphology for simulation
        self.importedcell = h.Import3d_GUI(nl,0)
        self.importedcell.instantiate(None) # Executes the connect statements and loads the cell into h scope

        # Create python lists from the morphology
        self.somatic = []
        self.dendritic = []
        self.all = []
        for sec in h.allsec():
            if 'soma' in sec.name():
                self.somatic.append(sec)
            if 'dend' in sec.name():
                self.dendritic.append(sec)
                
    def define_biophysics(self):
        """
        Distributes passive mechanisms and three different types
        of ion channels on the soma and dendrites
        """
        
        # Define passive properties
        for sec in self.all:
            sec.insert("pas")
            sec.g_pas = 0.003
            sec.e_pas = -65
            sec.ra = 173
            sec.cm = 1
            sec.ek = -90
            sec.ena = 55
            
        for sec in self.somatic:
            sec.insert('TC_iT')
            sec.pcabar_TC_iT = 5e-5
            sec.insert('TC_HH')
            sec.gk_max_TC_HH = 0.1
            sec.gna_max_TC_HH = 0.04
            sec.insert('TC_ih')
            sec.gh_max_TC_ih = 0.8e-5
            sec.insert('TC_iA')
            sec.gk_max_TC_iA = 0.01
   
        for sec in self.dendritic:
            sec.insert('TC_iT')
            sec.pcabar_TC_iT = 1.1e-5   
            sec.insert('TC_ih')
            sec.gh_max_TC_ih = 0.5e-5

In [ ]:
# Instantiate a cell from the class TC_neuron
cell = TC_neuron()
cell.build_morphology()

This is another way of visualizing the morphology of a neuron.
In this case we use a package called [NEUROM.](https://developer.humanbrainproject.eu/docs/projects/neurom/0.0.3/readme.html)

In [ ]:
import neurom
import neurom.viewer
fig, ax = neurom.viewer.draw(neurom.load_neuron('destexhe_morpho.swc'))


Now, here it is you can try exercice 4.

In [ ]:
recordings = {}
recordings['time'] =  ___ # Initialize a neuron vector 
recordings['v_soma'] = ___ # Initialize a neuron vector

recordings['time'].record() # NEURON reference to time
recordings['v_soma'].record(cell.somatic[0](0.5)._ref_v)

iclamp = h.IClamp(0.5, cell.somatic[0])
# Set up a step current with amplitude -0.1 nA, duration of 800 ms, starting after 200 ms from 
# the beginning of the simulation.


# Set up the simulation of 1500 ms. See examples above. 
# It's usual to set up the initial voltage close to steady state membrane potential (do you remember which is
# the parameter that influence it?). 
# Use a temperature of 34 celsius.
# Don't forget to inizialize and run the simulation


plt.plot(___) # Plot the results of the current-clamps experiment


___
### Futher interesting resources about NEURON

__The NEURON forum__: 
http://www.neuron.yale.edu/phpBB/

__NEURON programmer's reference and documentation__: 
http://www.neuron.yale.edu/neuron/static/new_doc/index.html

__NEURON web page and installation instructions__:
http://www.neuron.yale.edu/neuron/

__ModelDB: database of existing NEURON models__:
https://senselab.med.yale.edu/modeldb/

__The NEURON book and other readings__:

Nicholas T. Carnevale and Michael L. Hines, "The NEURON book". Cambridge University Press, 2006

M.L. Hines and N.T. Carnevale. "Neuron simulation environment". Encyclopedia of computational neuroscience, Springer, 2014
